# Building and Deploying contianers 

This is an attempt to streamline building and deploying containers we'll use during the development process.  

This is meant to be a living document and should continue to be updated as we progress.

## Building and Pushing `tlm` with the default version

**Note**: The following commands assume you are running them from the `/scale/scale/tlm` directory

The `build.sh` script can be execute in several ways. 

### 1. Build and Push the resulting image to Dockerhub

```bash
./build.sh
```

### 2. Build only

```bash
./build.sh build
```

### 3. Push only

```bash
./build.sh push
```

## Building and Pushing with a specified version

The `build.sh` script has been updated to accept a `-v` paramter for the version you wish to build.  If no version is provided, it will default to the version in [.env.tlm](../../scale/scale/tlm/.env.tlm)

We could further parameterize the scirpt to specify the whole tag and repo if we want.

### 1. Build and push a specified version

In the example, the version is `latest`, but the version you choose is up to you.  You'll later need to specify that version for deploying your container. 

```bash
./build.sh -v latest
```

### 2. Build a specified version

```bash
./build.sh build -v latest
```

### 3. Push a specified version

```bash
./build.sh push -v latest
```

## Deploying your container

If deploying containers for testing purposes, it will be easiest to use `helm`.  For basic helm setup and commands, see [helm notebook](helm.ipynb) or the [charts README](../../charts/README.md). 

Assuming you've went through the previous section and built a `tlm-service` image with the version `latest`, you can deploy it to your cluster with the following command: 

**Note**: This assumes you are running from the `/charts` directory, but you could run it from anywhere as long as you give it the absolute or relative path to the `/charts/tlm` chart directory. 

```bash
helm upgrade --install tlm ./tlm --namespace cash-flow --set image.version=latest
```

## Example

**Note**: if you have a current instance of `tlm-service` started using the [manage.sh script](../../k8s/manage.sh), stop the current instance by running `./manage.sh stop`. 

If you have current instance running via helm chart, that is fine.  This will upgrade it to your new version. 

### Build a container with your updates

The version in the example is `latest`, but you can use whatever you want.

In [7]:
%%bash

cd ../../scale/scale/tlm

./build.sh -v latest

Sourcing .env file
Building image: johndurkin/scale:tlm-service-latest


DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/



Sending build context to Docker daemon  14.34kB
Step 1/10 : FROM python:3.9-slim
 ---> 9d8cb7037cd8
Step 2/10 : WORKDIR /app
 ---> Using cache
 ---> 65f4af5222a7
Step 3/10 : ENV PYTHONPATH="${PYTHONPATH}:/app:$PYTHONPATH"
 ---> Using cache
 ---> 7aff90489d3a
Step 4/10 : COPY main.py /app
 ---> Using cache
 ---> 6a2a2bbb8323
Step 5/10 : COPY tlm_service.py /app
 ---> Using cache
 ---> e0739c8298ec
Step 6/10 : COPY requirements.txt /app
 ---> Using cache
 ---> b95cff49e483
Step 7/10 : RUN pip install --upgrade pip
 ---> Using cache
 ---> 6530289ceafe
Step 8/10 : RUN pip install --no-cache-dir -r requirements.txt
 ---> Using cache
 ---> 432df993768d
Step 9/10 : EXPOSE 4317 4318 4319
 ---> Using cache
 ---> aa811256d81a
Step 10/10 : CMD ["python", "main.py"]
 ---> Using cache
 ---> 5281d6917da4
Successfully built 5281d6917da4
Successfully tagged johndurkin/scale:tlm-service-latest
Pushing image: johndurkin/scale:tlm-service-latest
The push refers to repository [docker.io/johndurkin/scale]


### Deploy container with your `image.version`

This command will upgrade a current `tlm` chart if already installed, otherwise it will just install it 

In [8]:
%%bash

# go to the charts directory
cd ../../charts

helm upgrade --install tlm ./tlm --namespace cash-flow --set image.version=latest

Release "tlm" does not exist. Installing it now.
NAME: tlm
LAST DEPLOYED: Mon Oct 14 21:59:32 2024
NAMESPACE: cash-flow
STATUS: deployed
REVISION: 1
TEST SUITE: None


### Deploy container setting the `image.version` and `image.pullPolicy`

The default `pullPolicy` is **Always**.  These values are defined in the chart's [values.yaml](../../charts/tlm/values.yaml). 

You can override any of them.  Here is an example of setting both the `version` and `pullPolicy` for `tlm`

```bash
helm upgrade --install tlm ./tlm \
  --set image.version=latest \
  --set image.pullPolicy=IfNotPresent
```

In [12]:
%%bash

# go to the charts directory
cd ../../charts

helm upgrade --install tlm ./tlm \
  --set image.version=latest \
  --set image.pullPolicy=IfNotPresent

Release "tlm" does not exist. Installing it now.
NAME: tlm
LAST DEPLOYED: Mon Oct 14 22:36:01 2024
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None


### Verify your version is being used

In [10]:
%%bash

kubectl describe pod -l app=tlm-service -n cash-flow | grep image

  Normal  Pulling    22s   kubelet            Pulling image "johndurkin/scale:tlm-service-latest"
  Normal  Pulled     22s   kubelet            Successfully pulled image "johndurkin/scale:tlm-service-latest" in 598ms (598ms including waiting). Image size: 302494479 bytes.


## Deploying `otel-collector`

In [11]:
%%bash 

# go to the charts directory
cd ../../charts

helm upgrade --install otel-collector ./otel-collector --namespace cash-flow 

Release "otel-collector" does not exist. Installing it now.
NAME: otel-collector
LAST DEPLOYED: Mon Oct 14 22:02:38 2024
NAMESPACE: cash-flow
STATUS: deployed
REVISION: 1
TEST SUITE: None
